In [1]:
### Import PySpark and read input file as a DataFrame
from pyspark.sql import SparkSession
import sys

In [2]:
# create sparkSession instance
spark = SparkSession.builder.appName("HW4").getOrCreate()

# remove INFO logging from executing spark-session job on terminal
spark.sparkContext.setLogLevel("ERROR")

input_path = r"C:\Users\Justin_Hurcombe.PF164PM6\MSIS2427-BigData\whitehouse_waves-2016_12.csv" # input csv file



# create df from input path
df = spark.read.format("csv").option("header","true").option("inferSchema","true").load(input_path)


In [3]:
# convert all column headers to lowercase
for col in df.columns:
    df = df.withColumnRenamed(col,col.lower())

In [4]:
# create temporary view of df1 to execute SQL query
df.createOrReplaceTempView("df1")

In [5]:
# SQL query to create df2, where all missing namelast ans visitee_namelast values are removed
df2 = spark.sql("SELECT * FROM df1 WHERE (namelast IS NOT NULL AND visitee_namelast IS NOT NULL)")

In [6]:
### create visitor and visitee columns in df2 DataFrame following the (lastname, firstname) format


# import necessary packages
from pyspark.sql.functions import lower, concat, lit


# add the concatenated columns to df2
df2 = df2.withColumn('visitor', concat(lower(df.namelast),lit(', '),lower(df.namefirst)))

df2 = df2.withColumn('visitee',concat(lower(df.visitee_namelast),lit(', '),lower(df.visitee_namefirst)))

In [7]:
### group visitors (names are lower case) by frequency, and then show the top 10 most frequent visitors

# create dataframe to group visitors by count, and then order the results descending
visitor_count = df2.groupBy('visitor').count().orderBy("count",ascending=False)

# show top 10 rows of visitor dataframe 
print("Top 10 Most Frequent Visitors")
visitor_count.show(10)

Top 10 Most Frequent Visitors
+-----------------+-----+
|          visitor|count|
+-----------------+-----+
|  kidwell, lauren|  222|
| thomas, benjamin|  196|
|     haro, steven|  183|
|berner, katherine|  177|
|   grant, patrick|  155|
|     haas, jordan|  152|
|    garza, steven|  127|
|  martin, kathryn|  122|
|     cohen, mandy|  122|
|  brown, jennifer|  117|
+-----------------+-----+
only showing top 10 rows



In [8]:
### group visitees (names are lower case) by frequency, and then show the top 10 most frequently visited people

# create dataframe to group visitees by count, and then order the results descending
visitee_count = df2.groupBy('visitee').count().orderBy("count",ascending=False)

# show top 10 rows of visitee dataframe 
print("Top 10 Most Frequent Visitees")
visitee_count.show(10)

Top 10 Most Frequent Visitees
+--------------------+------+
|             visitee| count|
+--------------------+------+
|    office, visitors|430881|
|waves, visitorsof...| 44129|
|        bryant, ruth| 13970|
|       oneil, olivia| 13155|
|     thompson, jared| 11618|
|            /, potus| 10900|
|      burton, collin|  9672|
|      megan, matthew|  7944|
|     mayerson, asher|  6886|
| dessources, kalisha|  5289|
+--------------------+------+
only showing top 10 rows



In [9]:
### group visitor/visitee combos by frequency, and then show the top 10 most frequent visitor-visitee combo

# create dataframe to group visitor/visitee combos by count, and then order the results descending
visitor_visitee_count = df2.groupBy(['visitor','visitee']).count().orderBy("count",ascending=False)

# show top 10 rows of visitor/visitee combo dataframe
visitor_visitee_count.show(10)

+-----------------+---------------+-----+
|          visitor|        visitee|count|
+-----------------+---------------+-----+
|  kidwell, lauren| yudelson, alex|  103|
|     haas, jordan| yudelson, alex|   90|
|   grant, patrick| yudelson, alex|   89|
| thomas, benjamin| yudelson, alex|   89|
|     cohen, mandy|lambrew, jeanne|   84|
|     haro, steven| yudelson, alex|   84|
|berner, katherine| yudelson, alex|   82|
|   roche, shannon| yudelson, alex|   70|
| urizar, jennifer| johnson, katie|   68|
|  martin, kathryn|lambrew, jeanne|   61|
+-----------------+---------------+-----+
only showing top 10 rows



In [10]:
# SQL query to show count of records dropped from table df1
spark.sql("SELECT COUNT(*) AS Dropped_Records FROM df1 WHERE (namelast IS NULL OR visitee_namelast IS NULL)").show()

+---------------+
|Dropped_Records|
+---------------+
|          59255|
+---------------+



In [11]:
spark.stop()